In [1]:
from os import listdir
from os.path import isfile, join


#apk_path = 'APKs'
apk_path = '6test'
#sesName = apk_path + ".ag"
useSes = True ;

apks = []
for f in listdir(apk_path):
    if isfile(join(apk_path, f)) and f.endswith(".apk"):
        full = './' + apk_path + '/' + f
        apks.append(full)
        print("added: " + full)
        

#apks = [f for f in listdir(apk_path) if (isfile(join(apk_path, f)) and f.endswith(".apk"))]

print("number apk's found: ", len(apks))

added: ./6test/neg_a.duMobile.srv.apk
added: ./6test/neg_abhishek.arcdroid.tenth.apk
added: ./6test/neg_air.bg.teracomm.vivablitz.apk
added: ./6test/pos_a.a.hikidashi.apk
added: ./6test/pos_adsindy.com.terla.apk
added: ./6test/pos_air.A3bibe.apk
number apk's found:  6


In [2]:
from androguard import misc
from androguard import session
from os import listdir
from os.path import isfile, join

if useSes:
    s = misc.get_default_session()
    for apk in apks:    
        digest = s.add(apk)
        print("analysed: " + apk)
else:
    print("not using session analysis cache");


analysed: ./6test/neg_a.duMobile.srv.apk
analysed: ./6test/neg_abhishek.arcdroid.tenth.apk
analysed: ./6test/neg_air.bg.teracomm.vivablitz.apk
analysed: ./6test/pos_a.a.hikidashi.apk
analysed: ./6test/pos_adsindy.com.terla.apk
analysed: ./6test/pos_air.A3bibe.apk


In [4]:
from androguard.core.analysis.analysis import ExternalMethod

def analyseMethod(ma, level=0):    
    for pca, pm, offset in ma.get_xref_from(): #(ClassAnalysis, EncodedMethod or ExternalMethod, int
        #print(repr(ma.get_xref_from()))
        print("L{} Call from {} => {}:{}".format(level, pca.name, pm.name, offset))
        #print(repr(pm))
        #goto into recursion
        if(False): #doesnt work
            analyseMethod(pm,level+1) 

for apk in apks:
    if(useSes):
        a, d, dx = s.get_objects_apk(filename = apk)
    else:
        a, d, dx = misc.AnalyzeAPK(apk)        
    #print("processing: " + apk);
    
    #iterate over MethodClassAnalysis 
    for ma in dx.find_methods(classname="Landroid/webkit/WebSettings;", methodname='setUserAgentString'):        
        #m = ma.get_method()  
        #refs = ma.get_xref_from()
        #print("info {}".format(refs));
        # isExternalMethod = isinstance(m, ExternalMethod) # ? True : False
        #print("Found Method --> {}".format(m))
        analyseMethod(ma)
    

    
    

L0 Call from Lcom/google/ads/util/AdUtil; => a:32
L0 Call from Lcom/google/android/gms/internal/gj; => a:8
L0 Call from Lcom/google/android/gms/internal/dq; => a:8
